## Public Demo of IncQuery Server Jupyter Client Extensions

### Preliminaries

#### Setup IQS Connection

Execute the following piece of code to connect to the public IncQuery Server demo instance.

If you have non-guest privileges, you may specify your credentials here.

In [ ]:
import iqs_jupyter
iqs = iqs_jupyter.connect(
    address='https://openmbee.incquery.io/api',
    user='guest',
    password='incqueryserverguest'
)

#### Select MMS commit to consider

Run the next code block to display the commit selector widget, and use it to browse around the MMS repository. When you've had your fun, make sure to leave it in a state where a commit is selected from the _IQS4MMS Demos_ org, as we have made sure to pre-index and load those commits in the IQS.

In [2]:
commit_selector = iqs.jupyter_tools.mms_commit_selector_widget()

The following piece of code assigns the Python name `model` to the MMS commit selected above, and checks whether the model is indeed indexed and loaded by IQS, which is required for the rest of the demo 

In [3]:
model = commit_selector.value().to_model_compartment()
if model.is_loaded_by_server(iqs):
    print("We may proceed.")
else: 
    print("Model is not indexed&loaded by IQS, so the next demo steps will not work.")
    print(" (Unfortunately, guest users are not allowed to control model indexing.)")
    print("Please select another model from the 'IQS4MMS Demos' org.")

We may proceed.


### Model validation

#### Perform validation checks with custom validation rules (progress with indexed models only)

Request and display a validation report for the selected model

In [4]:
validation_report = iqs.validation.validate_model_compartment(model)
validation_report

{'compartment': {'compartment_uri': 'mms-index:/orgs/9ff6af30-af8a-4f9d-a26b-499010ba5b6e/projects/PROJECT-0e791c0e-16fe-422f-8f85-462ab035ce99/refs/master/commits/0e4c90e2-ee9d-4d7e-9d90-e7fb7bb2c0c8'},
 'diagnostics': {'debug': 0,
                 'error': 124,
                 'fatal': 15,
                 'info': 53,
                 'warning': 153},
 'rules': [{'constraint': {'element': {'compartment_uri': 'mms-index:/orgs/9ff6af30-af8a-4f9d-a26b-499010ba5b6e/projects/PROJECT-0e791c0e-16fe-422f-8f85-462ab035ce99/refs/master/commits/0e4c90e2-ee9d-4d7e-9d90-e7fb7bb2c0c8',
                                       'relative_element_id': '_19_0_85c027f_1531402451790_283399_173867'},
                           'name': 'Signal_Missing_From_Properties_Rule',
                           'type': 'constraint'},
            'matching_elements': [{'element': {'compartment_uri': 'mms-index:/orgs/9ff6af30-af8a-4f9d-a26b-499010ba5b6e/projects/PROJECT-0e791c0e-16fe-422f-8f85-462ab035ce99/refs/master/commits/0e4c90e2-ee9d-4d7e-9d90-e7fb7bb2c0c8',
                                               'relative_element_id': '_17_0_2_3_41e01aa_1380812166193_672423_45555'},
                                   'name': 'UNDEFINED',
                                   'type': 'com.nomagic.uml2.ext.magicdraw.commonbehaviors.mdcommunications.Signal'},
                                  {'element': {'compartment_uri': 'mms-index:/orgs/9ff6af30-af8a-4f9d-a26b-499010ba5b6e/projects/PROJECT-0e791c0e-16fe-422f-8f85-462ab035ce99/refs/master/commits/0e4c90e2-ee9d-4d7e-9d90-e7fb7bb2c0c8',
                                               'relative_element_id': '_17_0_2_3_41e01aa_1384359177966_579718_61418'},
                                   'name': 'UNDEFINED',
                                   'type': 'com.nomagic.uml2.ext.magicdraw.compositestructures.mdports.Port'},
                                  {'element': {'compartment_uri': 'mms-index:/orgs/9ff6af30-af8a-4f9d-a26b-499010ba5b6e/projects/PROJECT-0e791c0e-16fe-422f-8f85-462ab035ce99/refs/master/commits/0e4c90e2-ee9d-4d7e-9d90-e7fb7bb2c0c8',
                                               'relative_element_id': '_17_0_2_3_41e01aa_1383148297990_701142_55245'},
                                   'name': 'UNDEFINED',
                                   'type': 'com.nomagic.uml2.ext.magicdraw.commonbehaviors.mdcommunications.Signal'},
                                  {'element': {'compartment_uri': 'mms-index:/orgs/9ff6af30-af8a-4f9d-a26b-499010ba5b6e/projects/PROJECT-0e791c0e-16fe-422f-8f85-462ab035ce99/refs/master/commits/0e4c90e2-ee9d-4d7e-9d90-e7fb7bb2c0c8',
                                               'relative_element_id': '_17_0_2_3_41e01aa_1383542129649_120510_65768'},
                                   'name': 'UNDEFINED',
                                   'type': 'com.nomagic.uml2.ext.magicdraw.compositestructures.mdports.Port'},
                                  {'element': {'compartment_uri': 'mms-index:/orgs/9ff6af30-af8a-4f9d-a26b-499010ba5b6e/projects/PROJECT-0e791c0e-16fe-422f-8f85-462ab035ce99/refs/master/commits/0e4c90e2-ee9d-4d7e-9d90-e7fb7bb2c0c8',
                                               'relative_element_id': '_17_0_2_3_41e01aa_1384984608940_507845_64361'},
                                   'name': 'UNDEFINED',
                                   'type': 'com.nomagic.uml2.ext.magicdraw.commonbehaviors.mdcommunications.Signal'},
                                  {'element': {'compartment_uri': 'mms-index:/orgs/9ff6af30-af8a-4f9d-a26b-499010ba5b6e/projects/PROJECT-0e791c0e-16fe-422f-8f85-462ab035ce99/refs/master/commits/0e4c90e2-ee9d-4d7e-9d90-e7fb7bb2c0c8',
                                               'relative_element_id': '_17_0_2_3_41e01aa_1382560512974_563605_80855'},
                                   'name': 'UNDEFINED',
                                   'type': 'com.nomagic.uml2.ext.magicdraw.compositestructures.mdports.Port'},
                                  {'element': {'com

#### Visualize custom validation reports via Pandas dataframes and Plot.ly

The following block renders results in Pandas dataframe format, ready for complex client-side manipulation:

In [5]:
validation_report_df = validation_report.to_data_frame()

Setting up Cufflinks for Pandas/Plot.ly visualization:

In [6]:
import cufflinks as cf
cf.go_offline()

Preprocess results using Pandas and visualize the output using Plot.ly; this is handy for generating many kinds of custom validation reports.

First, here is a diagram showing the number of violations found for each validation rule:

In [7]:
validation_report_df.groupby(by=['constraint_element_name','severity']).size().unstack(level=-1).iplot( 
    kind='bar', barmode='stack', colors = iqs_jupyter.validation_color_scale,
    filename='cufflinks/categorical-bar-chart/frequent_diagnostic_rules',
    yTitle='Number of Diagnostics per Diagnostic Rule', title='Diagnostic Report',
)

Next, we do some precomputation with Pandas and plot a chart of violating elements against their total number of violations; if there are multiple severities (e.g. warnings and errors), the bars will be subdivided (stacked bar chart).

In [8]:
validation_report_elementwise = validation_report_df.groupby(by=['matching_element_relative_id','severity']).size().unstack(level=-1, fill_value=0)
validation_report_elementwise['total'] = validation_report_elementwise.apply(sum, axis=1)
cols_sorted = ['total'] + [severity for severity in iqs_jupyter.validation_diagnostics_ranking if severity in validation_report_elementwise.columns.to_list()]
validation_report_elementwise = validation_report_elementwise.sort_values(by=cols_sorted, ascending=False).drop('total', axis=1)

In [9]:
validation_report_elementwise.iplot( 
    kind='bar', barmode='stack', colors = iqs_jupyter.validation_color_scale,
    filename='cufflinks/stacked-bar-chart/diagnostics_per_element',
    yTitle='Number of Diagnostics per Violating Element', title='Detailed Diagnostic Report',
)

### Custom model queries

#### Execute pre-registered query and process results

See which queries are registered and ready for execution:

In [10]:
iqs.queries.list_queries()

{'query_fq_ns': ['iqs4mms.demo.benchmark.alphabeticalDependencies',
                 'iqs4mms.demo.benchmark.blocks',
                 'iqs4mms.demo.benchmark.blocksOrRequirements',
                 'iqs4mms.demo.benchmark.blocksOrRequirementsOrConstraints',
                 'iqs4mms.demo.benchmark.circularDependencies',
                 'iqs4mms.demo.benchmark.classes',
                 'iqs4mms.demo.benchmark.complexNameOfElements',
                 'iqs4mms.demo.benchmark.dependencies',
                 'iqs4mms.demo.benchmark.dependencyChains',
                 'iqs4mms.demo.benchmark.elementsWithShortNames',
                 'iqs4mms.demo.benchmark.incomingTransitions',
                 'iqs4mms.demo.benchmark.loopTransitionWithTriggerEffectEventNoGuard',
                 'iqs4mms.demo.benchmark.namedElement',
                 'iqs4mms.demo.benchmark.namesOfDependencyEndpoints',
                 'iqs4mms.demo.benchmark.nonBlockClasses',
                 'iqs4mms.demo.benchmark.numberOfSubregions',
                 'iqs4mms.demo.benchmark.numberOfTransitiveSubstates',
                 'iqs4mms.demo.benchmark.parentState',
                 'iqs4mms.demo.benchmark.requirements',
                 'iqs4mms.demo.benchmark.stateWithMostSubstates',
                 'iqs4mms.demo.benchmark.statesWithMoreIncomingTransitions',
                 'iqs4mms.demo.benchmark.statesWithShortNames',
                 'iqs4mms.demo.benchmark.subregions',
                 'iqs4mms.demo.benchmark.transitionPointingOutOfCompState',
                 'iqs4mms.demo.benchmark.transitiveSubstates',
                 'iqs4mms.demo.benchmark.transitiveSubstatesWithCheck',
                 'iqs4mms.demo.benchmark.transitiveSubstatesWithCheck2',
                 'iqs4mms.demo.benchmark.transitiveSubstatesWithCheck3',
                 'iqs4mms.demo.tmt.activity.PinWithoutConnectedParameters',
                 'iqs4mms.demo.tmt.activity.UnconnectedPins',
                 'iqs4mms.demo.tmt.activity.circularActivityCall',
                 'iqs4mms.demo.tmt.portsignal.ReceivedSignalNotInFlowProperties',
                 'iqs4mms.demo.tmt.portsignal.SentSignalNotInFlowProperties',
                 'iqs4mms.demo.tmt.signal.receivedButNotSentSignals',
                 'iqs4mms.demo.tmt.signal.sentButNotReceivedSignals',
                 'iqs4mms.demo.tmt.state.deadlockState',
                 'iqs4mms.demo.tmt.state.deadlockStateSimplified']}

Execute a custom query registered on the server and display its results. 

Below we demonstrate a query that assigns a pair of in-house custom requirements coverage metrics to packages in the model. 

(Towards the end of the notebook we will see how such custom queries can be defined and registered on the server.)

In [11]:
from iqs_jupyter import schema
qResults = iqs.query_execution.execute_query_on_model_compartment(
    schema.ExecuteQueryOnCompartmentRequest(
      model_compartment = model,
      query_fqn = "iqs4mms.demo.coverage.packageCoverage"
    ))
qResults

{'binding': [],
 'match_set_size': 754,
 'matches': [{'arguments': [{'parameter': 'pack',
                             'value': {'compartmentURI': 'mms-index:/orgs/9ff6af30-af8a-4f9d-a26b-499010ba5b6e/projects/PROJECT-0e791c0e-16fe-422f-8f85-462ab035ce99/refs/master/commits/0e4c90e2-ee9d-4d7e-9d90-e7fb7bb2c0c8',
                                       'relativeElementID': '_18_0_5_baa02e2_1454282952381_102699_153211'}},
                            {'parameter': 'totalBlocks', 'value': 0},
                            {'parameter': 'stronglyCovered', 'value': 0},
                            {'parameter': 'weaklyCovered', 'value': 0}]},
             {'arguments': [{'parameter': 'pack',
                             'value': {'compartmentURI': 'mms-index:/orgs/9ff6af30-af8a-4f9d-a26b-499010ba5b6e/projects/PROJECT-0e791c0e-16fe-422f-8f85-462ab035ce99/refs/master/commits/0e4c90e2-ee9d-4d7e-9d90-e7fb7bb2c0c8',
                                       'relativeElementID': 'HH_21_PROJECT-39602c53-e22f-4069-b071-8340d908fb0f'}},
                            {'parameter': 'totalBlocks', 'value': 0},
                            {'parameter': 'stronglyCovered', 'value': 0},
                            {'parameter': 'weaklyCovered', 'value': 0}]},
             {'arguments': [{'parameter': 'pack',
                             'value': {'compartmentURI': 'mms-index:/orgs/9ff6af30-af8a-4f9d-a26b-499010ba5b6e/projects/PROJECT-0e791c0e-16fe-422f-8f85-462ab035ce99/refs/master/commits/0e4c90e2-ee9d-4d7e-9d90-e7fb7bb2c0c8',
                                       'relativeElementID': '_12_1_8740266_1172576734562_842583_161'}},
                            {'parameter': 'totalBlocks', 'value': 0},
                            {'parameter': 'stronglyCovered', 'value': 0},
                            {'parameter': 'weaklyCovered', 'value': 0}]},
             {'arguments': [{'parameter': 'pack',
                             'value': {'compartmentURI': 'mms-index:/orgs/9ff6af30-af8a-4f9d-a26b-499010ba5b6e/projects/PROJECT-0e791c0e-16fe-422f-8f85-462ab035ce99/refs/master/commits/0e4c90e2-ee9d-4d7e-9d90-e7fb7bb2c0c8',
                                       'relativeElementID': '_17_0_2_3_41e01aa_1377752889917_280313_49039'}},
                            {'parameter': 'totalBlocks', 'value': 0},
                            {'parameter': 'stronglyCovered', 'value': 0},
                            {'parameter': 'weaklyCovered', 'value': 0}]},
             {'arguments': [{'parameter': 'pack',
                             'value': {'compartmentURI': 'mms-index:/orgs/9ff6af30-af8a-4f9d-a26b-499010ba5b6e/projects/PROJECT-0e791c0e-16fe-422f-8f85-462ab035ce99/refs/master/commits/0e4c90e2-ee9d-4d7e-9d90-e7fb7bb2c0c8',
                                       'relativeElementID': '_18_0_5_baa02e2_1457913287410_234257_148150'}},
                            {'parameter': 'totalBlocks', 'value': 749},
                            {'parameter': 'stronglyCovered', 'value': 3},
                            {'parameter': 'weaklyCovered', 'value': 94}]},
             {'arguments': [{'parameter': 'pack',
                             'value': {'compartmentURI': 'mms-index:/orgs/9ff6af30-af8a-4f9d-a26b-499010ba5b6e/projects/PROJECT-0e791c0e-16fe-422f-8f85-462ab035ce99/refs/master/commits/0e4c90e2-ee9d-4d7e-9d90-e7fb7bb2c0c8',
                                       'relativeElementID': '_17_0_5_407019f_1348077490251_262720_12434'}},
                            {'parameter': 'totalBlocks', 'value': 0},
                            {'parameter': 'stronglyCovered', 'value': 0},
                            {'parameter': 'weaklyCovered', 'value': 0}]},
             {'arguments': [{'parameter': 'pack',
                             'value': {'compartmentURI': 'mms-index:/orgs/9ff6af30-af8a-4f9d-a26b-499010ba5b6e/projects/PROJECT-0e791c0e-16fe-422f-8f85-462ab035ce99/refs/master/commits/0e4c90e2-ee9d-4d7e-9d90-e7fb7bb2c0c8',
                                       'relativeElementID': 'HH_18_DD_2_MM_

#### Visualize query results using Pandas dataframes and Plot.ly

Setting up Cufflinks for Pandas/Plot.ly visualization:

In [33]:
import cufflinks as cf
cf.go_offline()

The following block query renders results in Pandas dataframe format, ready for complex client-side preprocessing:

In [34]:
qResults_df = qResults.to_data_frame()
qResults_df['pack'] = qResults_df['pack'].apply(lambda element: element.relative_element_id)
qResults_df.set_index('pack', inplace=True)
qResults_df['nonCovered'] = qResults_df['totalBlocks'] - qResults_df['stronglyCovered'] - - qResults_df['weaklyCovered'] 

Visualize the output using Plot.ly:

In [35]:
qResults_df.query('totalBlocks != 0').sort_values(by='totalBlocks', ascending=False).drop('totalBlocks', axis=1).iplot(
    kind='bar', barmode='stack', filename='cufflinks/categorical-bar-chart/coverage',
    yTitle='Blocks in Package Traced to Requirements', title='Coverage Report',
)

#### Extract individual model elements, execute queries with parameter bindings

Descriptors of individual model elements can be extracted into Python variables from query results...

In [18]:
first_result_element = qResults.to_list_of_matches()[0]['pack']
first_result_element

{'compartment_uri': 'mms-index:/orgs/9ff6af30-af8a-4f9d-a26b-499010ba5b6e/projects/PROJECT-0e791c0e-16fe-422f-8f85-462ab035ce99/refs/master/commits/0e4c90e2-ee9d-4d7e-9d90-e7fb7bb2c0c8',
 'relative_element_id': '_18_0_5_baa02e2_1454282952381_102699_153211'}

...alternatively, model element descriptors can be directly constructed using element identifiers:

In [19]:
some_element = model.get_element_in_compartment_by_id("_18_0_2_baa02e2_1421374069634_721435_78319") 
some_element

{'compartment_uri': 'mms-index:/orgs/9ff6af30-af8a-4f9d-a26b-499010ba5b6e/projects/PROJECT-0e791c0e-16fe-422f-8f85-462ab035ce99/refs/master/commits/0e4c90e2-ee9d-4d7e-9d90-e7fb7bb2c0c8',
 'relative_element_id': '_18_0_2_baa02e2_1421374069634_721435_78319'}

Model elements or simple values can be used as parameter bindings to restrict the requested results:

In [20]:
from iqs_jupyter import binding
qResults_restricted = iqs.query_execution.execute_query_on_model_compartment(
    schema.ExecuteQueryOnCompartmentRequest(
        model_compartment = model,
        query_fqn = "iqs4mms.demo.coverage.packageCoverage",
        parameter_binding = binding(pack=some_element)
    ))
qResults_restricted

,pack,totalBlocks,stronglyCovered,weaklyCovered
,element #_18_0_2_baa02e2_1421374069634_721435_78319,,,
0,element #_18_0_2_baa02e2_1421374069634_721435_78319,150,2,81


### Sandbox

## Extra section for privileged users
Do not forget to specify your privileged credentials at the top of the notebook, in the first code cell

### Repository management, indexing commits

Force the server to refresh its knowledge of commits in the repository:

In [ ]:
iqs.mms_repository.update_mms_repository()

Index another model from the repository, and then load the index into server memory:

In [ ]:
iqs.persistent_index.index_model_compartment(model)

In [ ]:
iqs.in_memory_index.load_model_compartment(model)

### Define and register custom ad-hoc queries

The custom model queries defined below will discover multiple forms of traceability to _Requirement_ elements, determine the coverage of _SysML_ _Blocks_, and aggregate coverage metrics for _Package_s containing these _Blocks_.

In [28]:
coverage_query_package = "iqs4mms.demo.coverage"
coverage_query_main = "iqs4mms.demo.coverage.packageCoverage"
coverage_query_code = '''

// SECTION 1: main query

/* 
 * Associates a UML package with 
 *  - the total number of transitively contained SysML Blocks, and 
 *  - the number of strongly and weakly covered blocks among them
 */
pattern packageCoverage(
    pack: Package, 
    totalBlocks: java Integer, 
    stronglyCovered: java Integer, 
    weaklyCovered: java Integer
) {
    totalBlocks     == count find blockInPackage(_, pack);
    stronglyCovered == count find stronglyCoveredBlockInPackage(_, pack);
    weaklyCovered   == count find weaklyCoveredBlockInPackage(_, pack);
}


// SECTION 2: custom in-house definitions for strong and weak coverage

/* 
 * Identifies elements that are strongly covered by a requirement.
 * This definition may be customized according to in-house concept of strong coverage.
 */
pattern stronglyCovered(element: NamedElement) {
    find util.sysml.sysmlRequirement_SatisfiedBy(_, element);
} or {
    find util.sysml.sysmlRequirement_VerifiedBy(_, element);
} or {
    find util.sysml.sysmlRequirement_TracedTo(_, element);
}
/* 
 * Identifies elements that are NOT strongly covered, 
 * but are reachable from a strongly covered element 
 * using one or more custom propagation steps
 */
pattern weaklyCovered(element: NamedElement) {
    find stronglyCovered(otherElement);
    find coveragePropagates+(otherElement, element);
    neg find stronglyCovered(element);
}
/* 
 * Defines custom coverage propagation steps, where 
 * coverage of a 'from' element automatically implies weak coverage of a 'to' element.
 * This definition may be customized according to in-house propagation rules.
 */
pattern coveragePropagates(from: NamedElement, to: NamedElement) {
    // from container to contained part
    Property.owner(part, from);
    Property.aggregation(part, ::composite);
    TypedElement.type(part, to);
} or { 
    // from general superclassifier / block to specific block
    Generalization.general(gen, from);
    Generalization.specific(gen, to);
}

// SECTION 3: helper queries to assemble a report on (transitive) package contents

/* 
 * Associates SysML Blocks with UML packages directly or indirectly containing them.
 */
pattern blockInPackage(block: Class, pack: Package) {
    find util.sysml.sysmlBlock(block, _);    
    Element.owner+(block, pack);
}
/* 
 * Associates strongly covered Blocks with UML packages directly or indirectly containing them.
 */
pattern stronglyCoveredBlockInPackage(block: Class, pack: Package) {
    find stronglyCovered(block);    
    find blockInPackage(block, pack);
}
/* 
 * Associates weakly covered Blocks with UML packages directly or indirectly containing them.
 */
pattern weaklyCoveredBlockInPackage(block: Class, pack: Package) {
    find weaklyCovered(block);    
    find blockInPackage(block, pack);
}


'''

Queries need to be registered on the server before they can be evaluated.

In [29]:
(
    # attempt to register query... 
    iqs.queries.register_queries_plain_text(coverage_query_code, query_package=coverage_query_package)
    # ... but only if not already registered
    if coverage_query_main not in iqs.queries.list_queries().query_fq_ns
    else "Query is already registered"
)    

'Query is already registered'

### Sandbox